PyTorchを使ってLSTMで文章分類を実装（バッチ対応）

#### データ準備
livedoorニュースコーパスの「ldcc-20140209.tar.gz」データをダウンロードします。
https://www.rondhuit.com/download.html

#### データフレーム作成

In [1]:
%load_ext lab_black

In [2]:
import os
from glob import glob
import pandas as pd
import linecache

# カテゴリを配列で取得
categories = [name for name in os.listdir("text") if os.path.isdir("text/" + name)]
print(categories)

['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


In [3]:
datasets = pd.DataFrame(columns=["title", "category"])
for cat in categories:
    path = "text/" + cat + "/*.txt"
    files = glob(path)
    for text_name in files:
        title = linecache.getline(text_name, 3)
        s = pd.Series([title, cat], index=datasets.columns)
        datasets = datasets.append(s, ignore_index=True)

# データフレームシャッフル
datasets = datasets.sample(frac=1).reset_index(drop=True)
datasets.head()

,title,category
0,伊集院光がラジオで「24時間テレビ」について“本音”を語る\n,topic-news
1,ハイスペックだけじゃない！？使い勝手の良いドコモスマホ「ARROWS X F-10D」の使い...,smax
2,無料で使えるデジタルフィルターや便利なスタンプ機能搭載！小技の効いたカメラアプリ「LINE ...,smax
3,明日は我が身、“闇”に飲まれる債務者たちの転落人生\n,movie-enter
4,目指すはローラ!? 誰でもタメ口キャラは独女に有効か\n,dokujo-tsushin


#### 形態素解析エンジン定義

In [4]:
import MeCab
import re

tagger = MeCab.Tagger("-Owakati")


def make_wakati(sentence):
    sentence = tagger.parse(sentence)
    sentence = re.sub(r"[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+", " ", sentence)
    sentence = re.sub(
        r"[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+",
        "",
        sentence,
    )
    wakati = sentence.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

#### 単語IDの辞書を定義

In [5]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>": 0})

for title in datasets["title"]:
    wakati = make_wakati(title)
    for word in wakati:
        if word in word2index:
            continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))

vocab size :  12944


#### 系列の長さを揃えてバッチでまとめる

In [6]:
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

cat2index = {}
for cat in categories:
    if cat in cat2index:
        continue
    cat2index[cat] = len(cat2index)


def sentence2index(sentence):
    wakati = make_wakati(sentence)
    return [word2index[w] for w in wakati]


def category2index(cat):
    return [cat2index[cat]]


index_datasets_title_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for title, category in zip(datasets["title"], datasets["category"]):
    index_title = sentence2index(title)
    index_category = category2index(category)
    index_datasets_title_tmp.append(index_title)
    index_datasets_category.append(index_category)
    if max_len < len(index_title):
        max_len = len(index_title)

# 系列の長さを揃えるために短い系列にパディングを追加
# 後ろパディングだと正しく学習できなかったので、前パディング
index_datasets_title = []
for title in index_datasets_title_tmp:
    for i in range(max_len - len(title)):
        title.insert(0, 0)  # 前パディング
    #     title.append(0)　# 後ろパディング
    index_datasets_title.append(title)

train_x, test_x, train_y, test_y = train_test_split(
    index_datasets_title, index_datasets_category, train_size=0.7
)

# データをバッチでまとめるための関数
def train2batch(title, category, batch_size=100):
    title_batch = []
    category_batch = []
    title_shuffle, category_shuffle = shuffle(title, category)
    for i in range(0, len(title), batch_size):
        title_batch.append(title_shuffle[i : i + batch_size])
        category_batch.append(category_shuffle[i : i + batch_size])
    return title_batch, category_batch

#### モデル定義
パディング文字列ももちろん埋め込む必要があるわけですが、<pad>は0ベクトルで埋め込み、学習の妨げにならないようにするために、nn.Embedding()にてpadding_idx=0を追加。
LSTMを定義する際、batch_first=Trueを指定すると、LSTMのインプットの形式がバッチサイズ × 文章の長さ × ベクトル次元数になる。こうしたほうが次元を操作する際にわかりやすい。

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        # embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores


# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)
# to(device)でモデルがGPU対応する
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()
# SGDからAdamに変更。特に意味はなし
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### 学習
1epoch毎に全バッチを学習させます。バッチごとに逆伝搬してパラメータ更新させてます。

In [8]:
losses = []
for epoch in range(100):
    all_loss = 0
    title_batch, category_batch = train2batch(train_x, train_y)
    for i in range(len(title_batch)):
        batch_loss = 0

        model.zero_grad()

        # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
        title_tensor = torch.tensor(title_batch[i], device=device)
        # category_tensor.size() = (batch_size × 1)なので、squeeze()
        category_tensor = torch.tensor(category_batch[i], device=device).squeeze()

        out = model(title_tensor)

        batch_loss = loss_function(out, category_tensor)
        batch_loss.backward()
        optimizer.step()

        all_loss += batch_loss.item()
    print("epoch", epoch, "\t", "loss", all_loss)
    if all_loss < 0.1:
        break
print("done.")

/tmp/ipykernel_1202/393135737.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  tag_scores = self.softmax(tag_space.squeeze())


epoch 0 	 loss 93.04149925708771
epoch 1 	 loss 62.45664310455322
epoch 2 	 loss 43.46033388376236
epoch 3 	 loss 28.61568146944046
epoch 4 	 loss 17.592857524752617
epoch 5 	 loss 10.297913916409016
epoch 6 	 loss 5.414330828934908
epoch 7 	 loss 2.8578354120254517
epoch 8 	 loss 1.7016379898414016
epoch 9 	 loss 1.0987466918304563
epoch 10 	 loss 0.7342332750558853
epoch 11 	 loss 0.5618548397906125
epoch 12 	 loss 0.46699896827340126
epoch 13 	 loss 0.3963501160033047
epoch 14 	 loss 0.3569310759194195
epoch 15 	 loss 0.31966969510540366
epoch 16 	 loss 0.29835499310866
epoch 17 	 loss 0.27902118349447846
epoch 18 	 loss 0.2592971900012344
epoch 19 	 loss 0.2477262222673744
epoch 20 	 loss 0.24968541599810123
epoch 21 	 loss 0.22629665036220104
epoch 22 	 loss 0.21266001695767045
epoch 23 	 loss 0.21197290293639526
epoch 24 	 loss 0.20655955182155594
epoch 25 	 loss 0.19902404095046222
epoch 26 	 loss 0.1980102310772054
epoch 27 	 loss 0.19202346372185275
epoch 28 	 loss 0.187508708

#### 予測
バッチ毎にまとめて予測。
前回と比べて精度が上がっているのは単語の埋め込み次元数を上げたからだと思われます。

In [9]:
test_num = len(test_x)
a = 0
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
print("predict : ", a / test_num)

/tmp/ipykernel_1202/393135737.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  tag_scores = self.softmax(tag_space.squeeze())


predict :  0.684591052869408
